In [1]:
from datasets import Dataset
import pandas as pd
import evaluate
import numpy as np
from transformers import EarlyStoppingCallback,AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding, AutoTokenizer, set_seed
import os
from sklearn.model_selection import train_test_split
from scipy.special import softmax
import argparse
import logging
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import wandb
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
import os
from torch.utils.data import Dataset
import torch
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, confusion_matrix
from imblearn.under_sampling import RandomUnderSampler
from datasets import Dataset
import numpy as np
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from textstat.textstat import textstatistics
import pandas as pd
import language_tool_python
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.metrics import precision_score, recall_score, roc_auc_score, confusion_matrix
import numpy as np
from itertools import combinations
from sklearn.ensemble import RandomForestClassifier

c:\Users\Ghiki\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

# Initialize the grammar checker
tool = language_tool_python.LanguageTool('en-US')

def extract_text_features(text):
    # Tokenize the text
    words = word_tokenize(text)
    sentences = sent_tokenize(text)
    word_count = len(words)
    sentence_count = len(sentences)

    # Lexical diversity
    unique_words = set(words)
    lexical_diversity = len(unique_words) / word_count

    # Average sentence length
    avg_sentence_length = word_count / sentence_count

    # Average word length
    avg_word_length = sum(len(word) for word in words) / word_count

    # Lexical density
    tags = nltk.pos_tag(words)
    content_words = [word for word, tag in tags if tag.startswith(('N', 'V', 'J', 'R'))]
    lexical_density = len(content_words) / word_count

    # Readability scores
    flesch_reading_ease = textstatistics().flesch_reading_ease(text)
    fog_index = textstatistics().gunning_fog(text)

    # Grammatical errors
    matches = tool.check(text)
    grammatical_errors = len(matches)

    # Create a DataFrame
    return pd.Series({
        'word_count': word_count,
        'sentence_count': sentence_count,
        'lexical_diversity': lexical_diversity,
        'avg_sentence_length': avg_sentence_length,
        'avg_word_length': avg_word_length,
        'lexical_density': lexical_density,
        'flesch_reading_ease': flesch_reading_ease,
        'gunning_fog_index': fog_index,
        'grammatical_errors': grammatical_errors
    })

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ghiki\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Ghiki\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
df= pd.read_json('datasets/subtaskB_train.jsonl', lines=True)
test_df= pd.read_json('datasets/subtaskB_dev.jsonl', lines=True)
# df=df.sample(50000)

from tqdm import tqdm
tqdm.pandas()
features_df = df['text'].progress_apply(extract_text_features)
features_df_test = test_df['text'].progress_apply(extract_text_features)

100%|██████████| 3000/3000 [03:09<00:00, 15.83it/s]


In [4]:
features_df['text']=df['text']
features_df_test['text']=test_df['text']
features_df['label']=df['label']
features_df_test['label']=test_df['label']

features_df.to_csv('datasets/features_df_extended.csv', index=False)
features_df_test.to_csv('datasets/features_df_test_extended.csv', index=False)

In [5]:
X_train=pd.read_csv('datasets/features_df_extended.csv')
X_test=pd.read_csv('datasets/features_df_test_extended.csv')
y_train=X_train['label']
y_test=X_test['label']

best_combination=['word_count', 'avg_sentence_length', 'avg_word_length', 'gunning_fog_index', 'grammatical_errors']

X_train.head(5)

,word_count,sentence_count,lexical_diversity,avg_sentence_length,avg_word_length,lexical_density,flesch_reading_ease,gunning_fog_index,grammatical_errors,text,label
0,487.0,36.0,0.451745,13.527778,3.741273,0.515400,79.60,8.47,9.0,Forza Motorsport is a popular racing game that...,1
1,813.0,68.0,0.311193,11.955882,3.728167,0.479705,83.56,6.83,1.0,Buying Virtual Console games for your Nintendo...,1
2,1041.0,90.0,0.270893,11.566667,4.130644,0.504323,76.22,6.38,4.0,Windows NT 4.0 was a popular operating system ...,1
3,942.0,67.0,0.318471,14.059701,4.093418,0.532909,76.32,6.42,3.0,How to Make Perfume\n\nPerfume is a great way ...,1
4,652.0,42.0,0.352761,15.523810,3.846626,0.461656,79.40,8.78,2.0,How to Convert Song Lyrics to a Song'\n\nConve...,1


In [6]:
X_train.shape

(71027, 11)

In [7]:
X_test.shape

(3000, 11)